<a href="https://colab.research.google.com/github/ju-sants/Desafios-DNC/blob/main/Desafio_6_Classifica%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Settings

## Libraries

In [ ]:
import pandas as pd

## Functions

## Load Data

In [ ]:
df = pd.read_excel('https://docs.google.com/spreadsheets/d/1HgAxhLc8F0olIbDCkPKxM-AXN33uhMTIrc8zrgkvY98/export?usp=xlsx')

# Data Understanding